In [419]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('/Users/Oller/Downloads/Datern.mplstyle')

import numpy as np
import pandas as pd

#Gradient colours
from matplotlib.colors import LinearSegmentedColormap
nodes = [0, 0.5,1.0]
color = ['#00ACF0','#ffffff','#A834FF']
cmap = LinearSegmentedColormap.from_list("", list(zip(nodes, color)))

%config InlineBackend.figure_format='retina'

In [420]:
df = pd.read_csv("/Users/Oller/Downloads/Kaggle_Datern_Training_Set.csv")

In [421]:
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15966 entries, 0 to 15965
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               15966 non-null  int64  
 1   Name             15964 non-null  object 
 2   Platform         15966 non-null  object 
 3   Year_of_Release  15720 non-null  float64
 4   Genre            15964 non-null  object 
 5   Publisher        15918 non-null  object 
 6   EU_Sales         15966 non-null  float64
 7   Global_Sales     15966 non-null  float64
 8   Critic_Score     7771 non-null   float64
 9   Critic_Count     7771 non-null   float64
 10  User_Score       9545 non-null   object 
 11  User_Count       7161 non-null   float64
 12  Developer        9608 non-null   object 
 13  Rating           9496 non-null   object 
dtypes: float64(6), int64(1), object(7)
memory usage: 1.7+ MB


In [422]:

df = pd.read_csv("/Users/Oller/Downloads/Kaggle_Datern_Training_Set.csv")


In [423]:
#replacing none values with the most common publisher
df['Publisher'] = df['Publisher'].replace(to_replace='Null',value = 'EA' )
#replacing none values with most common genre
df['Genre'] = df['Genre'].replace(to_replace='',value = 'Action' )
#dropping columns not needed in the model
df = df.drop(['Developer'], axis=1)
df = df.drop(['Year_of_Release'], axis =1)
df = df.drop(['Name'], axis=1)
df = df.drop(['User_Count'], axis=1)
df = df.drop(['Rating'], axis=1)
df=df.drop(['Publisher'], axis=1)

#filling in the empty critic score and critic count with the mean for those columns
df['Critic_Score'] = df['Critic_Score'].fillna(73.76776)
df['Critic_Count'] = df['Critic_Count'].fillna(33.96)

#replacing entries that were ' tbd' with the mean value
df['User_Score'] = df['User_Score'].replace(to_replace='tbd',value = '6.612587')
df['User_Score'] = df['User_Score'].fillna(6.612587)

#these columns had no genre so are dropped from the training set
df = df.drop([15601, 15659])

In [424]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15964 entries, 0 to 15965
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            15964 non-null  int64  
 1   Platform      15964 non-null  object 
 2   Genre         15964 non-null  object 
 3   EU_Sales      15964 non-null  float64
 4   Global_Sales  15964 non-null  float64
 5   Critic_Score  15964 non-null  float64
 6   Critic_Count  15964 non-null  float64
 7   User_Score    15964 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ MB


In [425]:


#encoding data for platform genre and rating
# Encoding Platform
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
df.iloc[:, 1] = labelencoder1.fit_transform(df.iloc[:, 1])


#encoding genre
labelencoder1 = LabelEncoder()
df.iloc[:, 2] = labelencoder1.fit_transform(df.iloc[:, 2])



In [426]:
#add a new column with EU_Sales * User_Score
df['User_Score']= df.User_Score.astype(float)
#df['EU_Sales_x_Score'] = df['EU_Sales']* df['User_Score']
#this didn't work so didnt include it in the regression
print(df.head())
df.info()

    ID  Platform  Genre  EU_Sales  Global_Sales  Critic_Score  Critic_Count  \
0  184        16     10      0.26          5.23      94.00000         29.00   
1  378        16     10      2.36          3.49      84.00000         20.00   
2  457        26      0      0.97          3.06      74.00000         17.00   
3  476        16      2      1.02          3.00      73.76776         33.96   
4  610         0      8      0.14          2.53      73.76776         33.96   

   User_Score  
0    8.500000  
1    6.400000  
2    7.900000  
3    6.612587  
4    6.612587  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15964 entries, 0 to 15965
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            15964 non-null  int64  
 1   Platform      15964 non-null  int32  
 2   Genre         15964 non-null  int32  
 3   EU_Sales      15964 non-null  float64
 4   Global_Sales  15964 non-null  float64
 5   Critic_Score  1

In [427]:
#separating into response matrix (x) and vector of targets(y)

#Vector of targets(Y) is the global sales column
Y = df.iloc[:,4]


#removing global sales from the response matrix
X = df.drop(['Global_Sales'], axis=1)
#removed ID from the model
X = X.drop(['ID'], axis=1)

print(X)


       Platform  Genre  EU_Sales  Critic_Score  Critic_Count  User_Score
0            16     10      0.26      94.00000         29.00    8.500000
1            16     10      2.36      84.00000         20.00    6.400000
2            26      0      0.97      74.00000         17.00    7.900000
3            16      2      1.02      73.76776         33.96    6.612587
4             0      8      0.14      73.76776         33.96    6.612587
...         ...    ...       ...           ...           ...         ...
15961         0      2      0.04      73.76776         33.96    6.612587
15962         0     10      0.03      73.76776         33.96    6.612587
15963         0      0      0.02      73.76776         33.96    6.612587
15964         0      3      0.02      73.76776         33.96    6.612587
15965         0      3      0.01      73.76776         33.96    6.612587

[15964 rows x 6 columns]


In [428]:
from sklearn.linear_model import LinearRegression
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X, Y)

LinearRegression()

In [429]:
X_test = pd.read_csv("/Users/Oller/Downloads/Kaggle_Datern_Test_Set.csv")

In [430]:


X_test_new = X_test.iloc[:, 0:10]

X_test_new = X_test_new.drop(['Name', 'Year_of_Release', 'Publisher'], axis=1)


print(X_test_new)
X_test_new['User_Score'] = X_test_new['User_Score'].replace(to_replace='tbd',value = 6.612587)
X_test_new['User_Score']= X_test_new.User_Score.astype(float)
#X_test_new['EU_Sales_x_Score'] = X_test_new['EU_Sales']* X_test_new['User_Score']

        ID Platform         Genre  EU_Sales  Critic_Score  Critic_Count  \
0    15707       DS          Misc      0.00           NaN           NaN   
1    15757      PS2  Role-Playing      0.01          53.0          17.0   
2    15798       DS          Misc      0.00           NaN           NaN   
3    15819       XB       Shooter      0.00          45.0          25.0   
4    15956       DS        Sports      0.00           NaN           NaN   
..     ...      ...           ...       ...           ...           ...   
748   8059      PS4      Fighting      0.12          59.0          30.0   
749   8093      PSV        Action      0.02          71.0           4.0   
750   8142     XOne       Shooter      0.07           NaN           NaN   
751   8145      PS4        Action      0.00          67.0          25.0   
752   8192      PS4     Adventure      0.05           NaN           NaN   

    User_Score  
0          NaN  
1          7.6  
2          NaN  
3            2  
4          NaN

In [431]:

#calculate mean critic score and user score, replace NaN with mean 
Critic_Score_mean = X_test_new[['Critic_Score']].mean()
print(Critic_Score_mean)


#repopulate the NaN values with the mean critic score
X_test_new['Critic_Score'] = X_test_new['Critic_Score'].fillna(73.76776)
print(X_test_new['Critic_Score'])


# calculate mean user score 
X_test_new['User_Score'] = X_test_new['User_Score'].fillna(6.612587)
print(X_test_new['User_Score'].head(10))

#convert 'tbd' to average
X_test_new['User_Score'] = X_test_new['User_Score'].replace(to_replace='tbd',value = 6.612587 )

#need to convert all values to float
X_test_new['User_Score']= X_test_new.User_Score.astype(float)

print(X_test_new['User_Score'])

Critic_Score    73.76776
dtype: float64
0      73.76776
1      53.00000
2      73.76776
3      45.00000
4      73.76776
         ...   
748    59.00000
749    71.00000
750    73.76776
751    67.00000
752    73.76776
Name: Critic_Score, Length: 753, dtype: float64
0    6.612587
1    7.600000
2    6.612587
3    2.000000
4    6.612587
5    6.612587
6    6.612587
7    6.612587
8    6.612587
9    6.612587
Name: User_Score, dtype: float64
0      6.612587
1      7.600000
2      6.612587
3      2.000000
4      6.612587
         ...   
748    7.800000
749    7.400000
750    6.612587
751    8.000000
752    7.100000
Name: User_Score, Length: 753, dtype: float64


In [432]:
#encode the test data
# columns platform genre and rating need encoding as they are categorical
labelencoder1 = LabelEncoder()
#encoding platform in test data
X_test_new.iloc[:, 1] = labelencoder1.fit_transform(X_test_new.iloc[:, 1])

#encoding genre
X_test_new.iloc[:, 2] = labelencoder1.fit_transform(X_test_new.iloc[:, 2])

#encoding rating
X_test_new.iloc[:, 5] = labelencoder1.fit_transform(X_test_new.iloc[:, 5])

print(X_test_new)
X_test_new = X_test_new.drop(['ID'], axis=1)



        ID  Platform  Genre  EU_Sales  Critic_Score  Critic_Count  User_Score
0    15707         1      3      0.00      73.76776           217    6.612587
1    15757         4      7      0.01      53.00000            13    7.600000
2    15798         1      3      0.00      73.76776           312    6.612587
3    15819        12      8      0.00      45.00000            21    2.000000
4    15956         1     10      0.00      73.76776           311    6.612587
..     ...       ...    ...       ...           ...           ...         ...
748   8059         6      2      0.12      59.00000            26    7.800000
749   8093         8      0      0.02      71.00000             0    7.400000
750   8142        13      8      0.07      73.76776            92    6.612587
751   8145         6      0      0.00      67.00000            21    8.000000
752   8192         6      1      0.05      73.76776           477    7.100000

[753 rows x 7 columns]


In [433]:
y_pred = regr.predict(X_test_new)
y_pred

array([ 8.51410606e-01,  1.40717655e-01,  1.19741965e+00, -1.93545638e-02,
        1.21496855e+00,  2.36512112e-02,  1.20829905e+00,  1.18649305e+00,
        1.18720658e+00,  1.19887543e+00,  5.33821092e-02, -9.10308933e-02,
        9.36950307e-02,  3.21730562e-01,  1.17859375e+00,  1.14252396e+00,
        1.15401826e+00,  1.16498893e+00,  4.26758555e-02,  9.58659893e-02,
        6.38895801e-02,  1.14318293e+00,  7.85299569e-02,  1.17551925e+00,
        1.15112230e+00,  1.19259085e+00,  1.12040894e+00,  1.62048836e+01,
        5.19985105e+00,  7.40923073e+00,  5.83011856e+00,  6.21058645e+00,
        5.03110114e+00,  5.91372006e+00,  2.07261495e+00,  3.81178403e+00,
        2.35918327e+00,  2.05554385e+00,  2.53450965e+00,  9.94072535e-01,
        3.22451410e+00,  6.22675027e-01,  2.40164025e+00,  1.60299733e+00,
        3.24604507e+00,  3.05938031e+00,  1.57093351e+00,  1.12571817e+00,
        2.77030894e+00,  1.90436847e-01,  1.54479669e+00,  1.97607951e+00,
        1.98004030e+00,  

In [416]:
#create new results dataframe
results_df = pd.DataFrame(y_pred)

In [417]:
print(results_df.head())
print(results_df.info())

          0
0  0.851411
1  0.140718
2  1.197420
3 -0.019355
4  1.214969
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       753 non-null    float64
dtypes: float64(1)
memory usage: 6.0 KB
None


In [418]:
output = pd.DataFrame()
output['ID'] = X_test['ID']
output['Expected'] = y_pred
output.to_csv('output6.csv', index = False)


27       95
29      172
30      246
31      290
32      326
       ... 
743    7812
745    7876
748    8059
749    8093
751    8145
Name: ID, Length: 333, dtype: int64


             0       ID
0    -0.099819      NaN
1    -0.092064      NaN
2     0.066011      NaN
3     6.746447      NaN
4    -0.118870      NaN
..         ...      ...
352   0.603566      NaN
353  20.507660  14342.0
354   1.095071      NaN
355   0.480422      NaN
356   1.821252      NaN

[357 rows x 2 columns]
